In [1]:
# loading and normalizing cifar10
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

import torch.nn as nn
import torch.nn.functional as F

In [2]:
# import cifar10 from keras
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

type(x_train)
type(y_train)

print(x_train.shape) # need to conver this to [50000, 3, 32, 32] for torch tensors
print(y_train.shape)

/Users/maxtian/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(50000, 32, 32, 3)
(50000, 1)


# add train_val split

In [3]:
# change to torch tensor
x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train)
x_test = torch.from_numpy(x_test)
y_test = torch.from_numpy(y_test)


print(x_train.size())
print(x_test.size())
print(x_train.type())
print(y_train.type())

torch.Size([50000, 32, 32, 3])
torch.Size([10000, 32, 32, 3])
torch.ByteTensor
torch.ByteTensor


In [4]:
#x_train = x_train.type('torch.DoubleTensor')
#x_test = x_test.type('torch.DoubleTensor')
#y_train = y_train.type('torch.DoubleTensor')
#y_test = y_test.type('torch.DoubleTensor')
x_train = x_train.float()
x_test = x_test.float()
y_train = y_train.long()
y_test = y_test.long()
y_train = y_train.squeeze(1) # labels need to be squeezed/unraveled
y_test = y_test.squeeze(1)

In [5]:
print(y_test)

tensor([ 3,  8,  8,  ...,  5,  1,  7])


In [6]:
# checking shape and type
print(x_train.size())
print(x_test.size())
print(y_train.size())
print(y_test.size())
print(x_train.type())
print(x_test.type())
print(y_train.type())
print(y_test.type())

torch.Size([50000, 32, 32, 3])
torch.Size([10000, 32, 32, 3])
torch.Size([50000])
torch.Size([10000])
torch.FloatTensor
torch.FloatTensor
torch.LongTensor
torch.LongTensor


In [7]:
#reshaping torch tensor
x_train = x_train.permute(0,3,1,2)
x_test = x_test.permute(0,3,1,2)


In [8]:
# check tensor shape
# should be [n, channels, x, y]
print(x_train.size())
print(x_test.size())

torch.Size([50000, 3, 32, 32])
torch.Size([10000, 3, 32, 32])


In [9]:
# change into torch dataset

from torch.utils.data import TensorDataset

train = TensorDataset(x_train, y_train)

test = TensorDataset(x_test, y_test)

In [10]:
# setting up a train and test loader
trainloader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test, batch_size=64, shuffle=False)


In [11]:
# creating a function that initializes weights
def init_weight_conv(m):
    if type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight) #he initialize, can use xavier instead
        m.bias.data.fill_(0.01) # optional bias
        
def init_weight_linear(m):
    if type(m) == nn.Linear:
        torch.nn.init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0.01)
    

In [16]:
# creating NN

# set zero padding to (k-1)/2 for stride = 1
# 


class convnet(nn.Module):
    def __init__(self):
        super(convnet, self).__init__()
        self.conv_block1 = nn.Sequential(
                            nn.Conv2d(3, 8, 5, padding=2), # [in-channel, out-channel(# of filters), kernel size (5,5)]
                            nn.ReLU(),
                            nn.MaxPool2d(2,2)
        )
        self.conv_block2 = nn.Sequential(
                            nn.Conv2d(8, 8, 5, padding = 2), # note that in-channel for the next layer is the same as out-channel from prev conv layer
                            nn.ReLU(),
                            nn.MaxPool2d(2,2)
        )
        self.fc1 = nn.Linear(512, 32) # 8*5*5 is flattened 8 filters and 5x5 kernel size
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 10)
    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = x.view(x.size(0), -1) # flatten
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x
        
net = convnet()



In [13]:
# apply initializers
net.apply(init_weight_conv)
net.apply(init_weight_linear)

convnet(
  (conv_block1): Sequential(
    (0): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block2): Sequential(
    (0): Conv2d(8, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=512, out_features=32, bias=True)
  (dropout): Dropout(p=0.5)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)

In [17]:
# define loss and optimizer
import torch.optim as optim
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate) #net.parameters

In [ ]:
# model training
n_epochs = 2

for epoch in range(n_epochs):
    running_loss = 0.0
    for i, train_data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = train_data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print loss per n minibatches
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0
    
    # for printing average loss every epoch
    print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(epoch+1, running_loss / len(trainloader)))
print('Finished Training')

In [18]:
# modified model training to keep track of train/val loss
n_epochs = 2

for epoch in range(n_epochs):
    running_loss = 0.0
    for i, train_data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = train_data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print loss per n minibatches
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0
    
    # keep track of loss in test dataset 
    correct = 0
    total = 0
    running_test_loss = 0.0
    with torch.no_grad():
        for test_data in testloader:
            test_images, test_labels = test_data
            test_outputs = net(test_images)
            test_loss = criterion(test_outputs, test_labels)
            running_test_loss += test_loss.item()
            _, predicted = torch.max(test_outputs.data, 1)
            total += test_labels.size(0)
            correct += (predicted == test_labels).sum().item()


    
    
    # for printing average loss every epoch
    print("===> Epoch {} Complete: Train Avg. Loss: {:.4f}".format(epoch+1, running_loss / len(trainloader)))
    print("===> Epoch {} Complete: Test Avg. Loss: {:.4f}".format(epoch+1, running_test_loss / len(testloader)))
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
print('Finished Training')

[1,    50] loss: 2.392
[1,   100] loss: 2.307
[1,   150] loss: 2.306
[1,   200] loss: 2.302
[1,   250] loss: 2.305
[1,   300] loss: 2.304
[1,   350] loss: 2.303
[1,   400] loss: 2.304
[1,   450] loss: 2.303
[1,   500] loss: 2.304
[1,   550] loss: 2.303
[1,   600] loss: 2.303
[1,   650] loss: 2.303
[1,   700] loss: 2.303
[1,   750] loss: 2.304
===> Epoch 1 Complete: Train Avg. Loss: 0.0943
===> Epoch 1 Complete: Test Avg. Loss: 2.3033
Accuracy of the network on the 10000 test images: 9 %
[2,    50] loss: 2.303
[2,   100] loss: 2.304
[2,   150] loss: 2.303
[2,   200] loss: 2.303
[2,   250] loss: 2.303
[2,   300] loss: 2.304
[2,   350] loss: 2.303
[2,   400] loss: 2.303
[2,   450] loss: 2.303
[2,   500] loss: 2.304
[2,   550] loss: 2.304
[2,   600] loss: 2.303
[2,   650] loss: 2.303
[2,   700] loss: 2.303
[2,   750] loss: 2.303
===> Epoch 2 Complete: Train Avg. Loss: 0.0942
===> Epoch 2 Complete: Test Avg. Loss: 2.3028
Accuracy of the network on the 10000 test images: 10 %
Finished Traini

In [ ]:
len(trainloader)
len(testloader)
print(loss)
print(running_loss)

In [19]:
# testing
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 10 %
